In [2]:
import pygsti
from pygsti.extras.interpygate import *

In [3]:
from pygsti.extras.interpygate import do_process_tomography, vec, unvec

In [4]:
from pygsti.tools.basistools import change_basis

In [5]:
from mpi4py import MPI
_comm = MPI.COMM_WORLD
_rank = _comm.Get_rank()
_size = _comm.Get_size()

In [6]:
import numpy as _np
from scipy.linalg import expm as _expm

In [7]:
from pygsti.extras.interpygate.core import PhysicalProcess, InterpolatedDenseOp, InterpolatedOpFactory

In [8]:
class ExampleProcess(PhysicalProcess):
    def __init__(self):
        import numpy as _np
        import scipy as _sp

        self.Hx = _np.array([[0, 0, 0, 0],
                            [0, 0, 0, 0],
                            [0, 0, 0, -1],
                            [0, 0, 1, 0]], dtype='float')
        self.Hy = _np.array([[0, 0, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 0, 0],
                            [0, -1, 0, 0]], dtype='float')
        self.Hz = _np.array([[0, 0, 0, 0],
                            [0, 0, -1, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 0]], dtype='float')

        self.dephasing_generator = _np.diag([0, -1, -1, 0])
        self.decoherence_generator = _np.diag([0, -1, -1, -1])
        num_params = 5  # omega, phase, detuning, dephasing, decoherence (does *not* include time)
        super().__init__(num_params)

    
    def advance(self, state, v, times):
        state = _np.array(state, dtype='complex')
        omega, phase, detuning, dephasing, decoherence = v

        H = (omega * _np.cos(phase) * self.Hx + omega * _np.sin(phase) * self.Hy + detuning * self.Hz)
        L = dephasing * self.dephasing_generator + decoherence * self.decoherence_generator

        processes = [change_basis(_expm((H + L) * t),'pp', 'col') for t in times]
        states = [unvec(_np.dot(process, vec(_np.outer(state, state.conj())))) for process in processes]

        return states

    def create_process_matrices(self, v, times=None, comm=None):                                                                                                                                                                                             
        print(f'Calling process tomography as {comm.Get_rank()} of {comm.Get_size()} on {comm.Get_name()}.')
        def state_to_process_mxs(state):
            return self.advance(state, v, times)
        processes = do_process_tomography(state_to_process_mxs, #opt_args={'v':v, 'times':times},
                                          n_qubits = 1, basis='pp', time_dependent=True, comm=comm)

        return processes
    
example_process = ExampleProcess()
target_mx = example_process.create_process_matrices(_np.array([1.0, 0.0, 0.0, 0.0, 0.0]), times=[_np.pi/2], comm=_comm)[0]
target_op = pygsti.obj.StaticDenseOp(target_mx)
print(target_op)
#target_mx[0]

Calling process tomography as 0 of 1 on MPI_COMM_WORLD.
StaticDenseOp with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0   0-1.00
   0   0 1.00   0



In [9]:
param_ranges = ([[0.9, 1.1], [-.1, .1], [-.1, .1], [0, 0.1], [0, 0.1]])
interp_orders = [2,2,2,2,2]
times = _np.linspace(_np.pi / 2, _np.pi / 2 + .5, 10)

interp_op = InterpolatedDenseOp.create_by_interpolating_physical_process(
    target_op, example_process, param_ranges, interp_orders, times, comm=_comm)

Group 0 processing 32 points on 1 processors.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1

In [10]:
# import pickle
# pickle.dump(interp_op, open("test.pkl",'wb'))
# interp_op2 = pickle.load(open("test.pkl",'rb'))
# interp_op2.to_dense()

In [11]:
print(interp_op.to_dense())
interp_op.from_vector([1.59, 1.1, 0.01, 0.01, 0.055, 0.055])
print(interp_op.to_vector())
interp_op.to_dense()

[[ 1.00000857e+00 -1.40563349e-06  5.84337416e-06  7.08736822e-06]
 [-2.33252786e-06  9.63076446e-01  6.06194750e-03 -7.50632085e-03]
 [ 3.39299600e-06 -1.41574751e-03 -4.69342197e-03 -9.63720772e-01]
 [-2.18470892e-06 -1.84824890e-04  9.64173068e-01  7.38404234e-03]]
[1.59  1.1   0.01  0.01  0.055 0.055]


array([[ 1.00000267e+00, -5.01756268e-06, -1.04592531e-06,
         4.14458334e-07],
       [ 5.35605128e-06,  9.56257800e-01,  7.57899819e-03,
         4.00061937e-03],
       [ 7.49717076e-07,  2.80221693e-03, -4.61850366e-02,
        -9.57731435e-01],
       [-9.00506113e-07, -3.79037305e-03,  9.59008435e-01,
        -4.69608952e-02]])

In [12]:
class TargetOpFactory(pygsti.obj.OpFactory):
    def __init__(self):
        self.process = ExampleProcess()
        pygsti.obj.OpFactory.__init__(self, dim=4, evotype="densitymx")
        
    def create_object(self, args=None, sslbls=None):
        assert(sslbls is None) # don't worry about sslbls for now -- these are for factories that can create gates placed at arbitrary circuit locations
        assert(len(args) == 2)  # t (time), omega
        t, omega = args
        mx = self.process.create_process_matrices(_np.array([omega, 0.0, 0.0, 0.0, 0.0]), times=[t], comm=_comm)[0]
        return pygsti.obj.StaticDenseOp(mx)

arg_ranges = [[0.9, 1.1]]  # a single arg (+time)
param_ranges = [[-.1, .1], [-.1, .1], [0, 0.1], [0, 0.1]]
interp_orders = [2,2,2,2,2]
times = _np.linspace(_np.pi / 2, _np.pi / 2 + .5, 10)

opfactory = InterpolatedOpFactory.create_by_interpolating_physical_process(
    TargetOpFactory(), example_process, arg_ranges, param_ranges, interp_orders, times, comm=_comm)

Group 0 processing 32 points on 1 processors.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process 

Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling pr

Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling pr

In [13]:
print(opfactory.num_params)
print(opfactory.to_vector())
opfactory.from_vector(_np.array([0.01, 0.01, 0.055, 0.055]))
op = opfactory.create_op((1.59, 1.1))
print(op.to_vector())
op.to_dense()

4
[0.   0.   0.05 0.05]
Calling process tomography as 0 of 1 on comm_world.
[0.01  0.01  0.055 0.055]


array([[ 1.00000096e+00,  7.56537629e-07,  1.35813351e-06,
        -4.08684549e-06],
       [-3.18018675e-07,  9.57580303e-01,  6.79116619e-03,
         2.51636076e-03],
       [-3.25786192e-07,  1.75755081e-03, -1.59676558e-01,
        -9.32426599e-01],
       [-3.46604509e-07, -3.05381978e-03,  9.54776234e-01,
        -1.64564095e-01]])

In [14]:
#print(op.target_op)

# Testing `do_process_tomography` subroutine

In [16]:
""" Demonstrate the process tomography function with (potentially) time-dependent outputs. """
sigI = _np.array([[1, 0], [0, 1]], dtype='complex')
sigX = _np.array([[0, 1], [1, 0]], dtype='complex')
sigY = _np.array([[0, -1.j], [1.j, 0]], dtype='complex')
sigZ = _np.array([[1, 0], [0, -1]], dtype='complex')
theta = .32723
u = _np.cos(theta) * sigI + 1.j * _np.sin(theta) * sigX
v = _np.sin(theta) * sigI - 1.j * _np.cos(theta) * sigX

U = _np.kron(u, v)
n_qubits = 2
# U = u
# n_qubits = 1
test_process = _np.kron(U.conj().T, U)


def single_time_test_function(pure_state, test_process=test_process):
    rho = vec(_np.outer(pure_state, pure_state.conj()))
    return unvec(_np.dot(test_process, rho))


def multi_time_test_function(pure_state, test_process=test_process):
    rho = vec(_np.outer(pure_state, pure_state.conj()))
    return [unvec(_np.dot(test_process, rho)), unvec(_np.dot(_np.linalg.matrix_power(test_process,2), rho))]


process_matrix = do_process_tomography(single_time_test_function, n_qubits=2, verbose=False)
if _rank == 0:
    test_process_pp = change_basis(test_process, 'col', 'pp')
    print("\nSingle-time test result should be True:")
    print(_np.isclose(process_matrix, test_process_pp).all())

process_matrices = do_process_tomography(multi_time_test_function, n_qubits=2, verbose=False, time_dependent=True)
if _rank==0:
    test_process = change_basis(test_process, 'col', 'pp')
    print("\nMulti-time test result should be [True, False]:")
    print([_np.isclose(x, test_process).all() for x in process_matrices])


Single-time test result should be True:
True

Multi-time test result should be [True, False]:
[True, False]


#  OLD VERSION

In [ ]:
#OLD (REMOVE LATER) 
# class ProcessFunction(object):
#     def __init__(self):
#         import numpy as _np
#         import scipy as _sp

#         self.Hx = _np.array([[0, 0, 0, 0],
#                             [0, 0, 0, 0],
#                             [0, 0, 0, -1],
#                             [0, 0, 1, 0]], dtype='float')
#         self.Hy = _np.array([[0, 0, 0, 0],
#                             [0, 0, 0, 1],
#                             [0, 0, 0, 0],
#                             [0, -1, 0, 0]], dtype='float')
#         self.Hz = _np.array([[0, 0, 0, 0],
#                             [0, 0, -1, 0],
#                             [0, 1, 0, 0],
#                             [0, 0, 0, 0]], dtype='float')

#         self.dephasing_generator = _np.diag([0, -1, -1, 0])
#         self.decoherence_generator = _np.diag([0, -1, -1, -1])

#     def advance(self, state, v = None, times = None):
#         state = _np.array(state, dtype='complex')
#         if times is None:
#             t, omega, phase, detuning, dephasing, decoherence = v
#             times = [t]
#         else:
#             omega, phase, detuning, dephasing, decoherence = v

#         H = (omega * _np.cos(phase) * self.Hx + omega * _np.sin(phase) * self.Hy + detuning * self.Hz)
#         L = dephasing * self.dephasing_generator + decoherence * self.decoherence_generator

#         processes = [change_basis(_expm((H + L) * t),
#                                                           'pp', 'col') for t in times]
#         states = [unvec(_np.dot(process, vec(_np.outer(state, state.conj())))) for process in processes]

#         return states

#     def __call__(self, v, times=None, comm=_comm):
#         print(f'Calling process tomography as {comm.Get_rank()} of {comm.Get_size()} on {comm.Get_name()}.')
#         processes = do_process_tomography(self.advance, opt_args={'v':v, 'times':times},
#                                           n_qubits = 1, basis='pp', time_dependent=True, comm=comm)

#         return processes

    
    
# gy = PhysicalProcess(mpi_workers_per_process=1, basis='col')
# gy.set_process_function(ProcessFunction(), mpi_enabled=True)
# target = change_basis(_np.array([[1,0,0,0],
#                                  [0,1,0,0],
#                                  [0,0,0,-1],
#                                  [0,0,1,0]], dtype='complex'), 'pp', 'pp')
# gy.set_target(target)

# # # # Evaluate one time point per run
# # # gy.set_parameter_range([[_np.pi/2-.5, _np.pi/2+.5],[0.9,1.1],[-.1,.1],[-.1,.1],[0,0.1],[0,0.1]])
# # # gy.set_interpolation_order([3,3,3,3,3,3])
# #
# # Evaluate many time points per run
# # Notice that the number of parameters listed below is 1 fewer than in the previous example
# gy.set_parameter_range([[0.9, 1.1], [-.1, .1], [-.1, .1], [0, 0.1], [0, 0.1]])
# gy.set_interpolation_order([3,3,3,3,3])
# gy.set_times(_np.linspace(_np.pi / 2, _np.pi / 2 + .5, 10))

# gy.interpolate()